# Finding most likely rhythmic patterns (Quantizing)

We would like to take the floating point tick values (in units of 'beats') and round them to some rational fraction that makes musical sense. We might imagine that there are different granular levels of musical complexity around a single beat: one could simply play four notes to a beat, or they could play twelve notes to that beat. Note that most of Western music is broken into fourths and/or thirds, and these sub-intervals may also be broken up themselves in similar ways. 

I propose a three-level hierarchy:

###Level 1: Thirds and Fourths
###Level 2: Same as Level 1 with Sixths and Eighths added
###Level 3: Same as Level 2 with Ninths, Twelvths, and Sixteenths added

It's unclear which of these quantizations will be the most useful for us in building our rhythm markov model, so let's build all three!

In [89]:
import pickle
from math import modf
from bisect import bisect

In [90]:
with open('../all_rhythms.pkl', 'r') as g:
    all_rhythms = pickle.load(g)

First, we'll construct some global lists for quantization and a general function that will round a given floating point tick value to the nearest quantized value.

In [102]:
level_lists = [[0.0, 1/4.0, 1/3.0, 1/2.0, 2/3.0, 3/4.0, 1.0], 
               [0.0, 1/8.0, 1/6.0, 1/4.0, 1/3.0, 3/8.0, 1/2.0, 
                5/8.0, 2/3.0, 3/4.0, 5/6.0, 7/8.0, 1.0],
               [0.0, 1/16.0, 1/12.0, 1/9.0, 1/8.0, 1/6.0, 3/16.0, 
                2/9.0, 1/4.0, 5/16.0, 1/3.0, 3/8.0, 5/12.0, 
                7/16.0, 4/9.0, 1/2.0, 5/9.0, 9/16.0, 7/12.0, 
                5/8.0, 2/3.0, 11/16.0, 3/4.0, 7/9.0, 13/16.0, 
                5/6.0, 7/8.0, 8/9.0, 11/12.0, 15/16.0, 1.0]]

def myround(tick, level):
    """Rounds a tick value to the nearest fractional beat interval 
    at the given level. The levels are defined as: 
    
    Level 1: Thirds and Fourths
    Level 2: Level 1 & Sixths and Eighths
    Level 3: Level 2 & Ninths, Twelvths, and Sixteenths
    
    Inputs: tick is a floating point tick value
            level is one of {1, 2, 3} as defined above
            
    Outputs: quantized tick value
    """
    
    if int(tick) == tick:
        return tick
    
    level_list = level_lists[level - 1]
    
    frac_tick, whole_tick = modf(tick)
    
    insert = bisect(level_list, frac_tick)
    
    dist1, dist2 = abs(frac_tick - level_list[insert - 1]), abs(frac_tick - level_list[insert])
    
    if dist1 < dist2:
        return whole_tick + round(level_list[insert - 1], 4)
    return whole_tick + round(level_list[insert], 4)

For example:

In [103]:
myround(6.9083333, 1), myround(6.9083333, 2), myround(6.9083333, 3)

(7.0, 6.875, 6.9167)

Now we can map the raw tick values into each grid using this helper function.

In [105]:
def quantize(level):
    all_rhythms_quantized = []
    for rhythm in all_rhythms:
        inner = []
        for tick in rhythm:
            new_tick = myround(tick, level)
            if len(inner) > 0 and new_tick == inner[-1]:
                continue
            inner.append(new_tick)
        all_rhythms_quantized.append(inner)
    assert len(all_rhythms_quantized) == len(all_rhythms)
    return all_rhythms_quantized

In [106]:
all_rhythms_level_1 = quantize(1)
all_rhythms_level_2 = quantize(2)
all_rhythms_level_3 = quantize(3)

Let's make sure this worked properly:

In [107]:
print all_rhythms_level_1[0]

[3.0, 4.0, 5.0, 5.75, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 14.6667, 14.75, 15.0, 16.0, 17.0, 171.0, 172.0, 173.0, 174.0]


In [108]:
print all_rhythms_level_2[0]

[3.0, 4.0, 5.0, 5.75, 5.875, 6.125, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 14.6667, 14.8333, 15.0, 16.0, 17.0, 171.0, 172.0, 173.0, 174.0]


In [109]:
print all_rhythms_level_3[0]

[3.0, 4.0, 5.0, 5.6875, 5.875, 6.0625, 6.9375, 8.0, 8.9375, 9.0, 10.0, 11.0, 11.9375, 12.0, 13.0, 14.0, 14.6875, 14.8333, 15.0, 16.0, 17.0, 171.0, 171.9375, 173.0, 174.0]


Success!

In [110]:
with open('../all_rhythms_level_1.pkl', 'w') as f:
    pickle.dump(all_rhythms_level_1, f)

In [111]:
with open('../all_rhythms_level_2.pkl', 'w') as f:
    pickle.dump(all_rhythms_level_2, f)

In [112]:
with open('../all_rhythms_level_3.pkl', 'w') as f:
    pickle.dump(all_rhythms_level_3, f)